# IMplementación del algoritmo Multiscale Morphological Contrast Enhancement (MMCE)

In [1]:
# Importación de librerías
import numpy as np
import cv2
from skimage.morphology import square, opening, closing

In [ ]:
# Primera versión de la función MMCE
def MMCE1(image, n):
    WTH = []
    BTH = []
    for i in range(1, n+1):
        se = square(2*i+1)
        WTH.append(image - opening(image, se))
        BTH.append(closing(image, se) - image)

    WTH_S = [WTH[i] - WTH[i-1] for i in range(1, n)]
    BTH_S = [BTH[i] - BTH[i-1] for i in range(1, n)]

    WTH_M = np.max(WTH, axis=0)
    BTH_M = np.max(BTH, axis=0)
    WTH_MS = np.max(WTH_S, axis=0)
    BTH_MS = np.max(BTH_S, axis=0)

    enhanced_image = image + (WTH_M + WTH_MS) - (BTH_M + BTH_MS)
    return enhanced_image

In [ ]:
# Segunda versión de la función MMCE
def MMC2(image, n_iterations):
    # Primera etapa del algoritmo: Obtener escalas de brillo y oscuridad
    scales = [cv2.getStructuringElement(cv2.MORPH_RECT, (2*i+1, 2*i+1)) for i in range(1, n_iterations+1)]
    WTH_scales = [cv2.subtract(image, cv2.morphologyEx(image, cv2.MORPH_TOPHAT, scale)) for scale in scales]
    BTH_scales = [cv2.subtract(cv2.morphologyEx(image, cv2.MORPH_BLACKHAT, scale), image) for scale in scales]
    
    # Segunda etapa del algoritmo: Obtener sustracciones de escalas
    WTH_diff_scales = [cv2.subtract(WTH_scales[i], WTH_scales[i-1]) for i in range(1, len(WTH_scales))]
    BTH_diff_scales = [cv2.subtract(BTH_scales[i], BTH_scales[i-1]) for i in range(1, len(BTH_scales))]
    
    # Tercera etapa del algoritmo: Calcular máximos valores
    WTH_max_scale = max(WTH_scales, key=np.max)
    BTH_max_scale = max(BTH_scales, key=np.max)
    WTH_diff_max_scale = max(WTH_diff_scales, key=np.max)
    BTH_diff_max_scale = max(BTH_diff_scales, key=np.max)
    
    # Etapa final: Obtener la imagen con aumento de contraste
    enhanced_image = cv2.add(cv2.subtract(cv2.add(image, WTH_max_scale), BTH_max_scale), cv2.subtract(WTH_diff_max_scale, BTH_diff_max_scale))
    
    return enhanced_image

In [ ]:
# Tercera versión de la función MMCE
def mmce3(image, n=3, gamma=1.2, beta=2.0):
  # Normalizar la imagen a valores entre 0 y 1
    
    """
  Implementación del algoritmo MMCE para mejorar el contraste de la imagen.

  Args:
    image: Imagen de entrada (en escala de grises).
    n: Número de iteraciones para la extracción de características multiescala.
    gamma: Parámetro que controla la intensidad del realce de contraste.
    beta: Parámetro que controla la suavidad del realce de contraste.

  Returns:
    Imagen mejorada.
  """
    image = image / 255.0
    # Aplicar la transformación top-hat multiescala
    enhanced_image = []
    for k in range(n):
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
        filtered_image = cv2.morphologyEx(image, cv2.MORPH_BLACKHAT, kernel)
        enhanced_image.append(filtered_image)

    # Combinar las características de imagen multiescala
    enhanced_image = np.array(enhanced_image).mean(axis=0)

    # Aplicar la función de transformación de contraste
    enhanced_image = np.power(enhanced_image, gamma) * beta
    # Limitar los valores entre 0 y 1
    enhanced_image = np.clip(enhanced_image, 0, 1)
    # Convertir la imagen mejorada al formato de salida
    enhanced_image = enhanced_image * 255.0
    enhanced_image = enhanced_image.astype(np.uint8)
    return enhanced_image


    


